In [107]:
from bitarray import bitarray #package that makes working with long bit arrays easier (pip install bitarray)

#Generate the WSPR Frequency Shift Key values (4FSK)

#Jamie Taylor KK6OLF
#January 2018

#This code was written to make the WSPR protocol encoding easily understandable.
#For that reason it emphasizes readable at the expense of some efficiency and also avoids
#some obvious Python idioms to make the code more understandable by non-Python readers

#This code is useful if you want to generate the 4FSK values for WSPR messages to drop in an embedded processor
#or adapt into a GNU Radio Python block to send WSPR using an SDR device (I've used it for both.)

#Note that in the convolutional encoding section the convolution polynomials (taps) get bit reversed
#this is so the code corresponds to a set of slides I created which step through the convolutional encoding process
#where the shift registers are move left to right (making them *appear* to move MSB to LSB with the message 
#fed MSB first into the shift register.) I'm not sure why, but this seems to be the standard pedagogical convention 
#and I thought it was easier to make my presentation comparable to other explanations of convolutional encoding 
#should people be looking at multiple instructional materials. (Of course the shifts are all relative, 
#as long are you are consistent - I just tried to write the code to look like most of the examples 
#people will run into.)

#Released under CCO - have fun with it.
#WSPR encoding in Python by Jamie Taylor
#To the extent possible under law, the person who associated CC0 with
#WSPR encoding in Python has waived all copyright and related or neighboring rights to WSPR encoding in Python.
#You should have received a copy of the CC0 legalcode along with this work.  
#If not, see <http://creativecommons.org/publicdomain/zero/1.0/>.

#The components of the message sent in the WSPR protocol
callsign = "KK6OLF"
gridsquare = "CM87" #https://en.wikipedia.org/wiki/Maidenhead_Locator_System (Why not use S2Cells?!)
powerlevel = 20

#For comparison see the encoded message for George Smart
#https://www.george-smart.co.uk/arduino/arduino_wspr/
#callsign = " M1GEO" #note the leading space to pad the call sign to 6 chars (3rd char must be number)
#gridsquare = "JO01"
#powerlevel = 20

In [108]:
#helper function for debugging - display a binary string of 1's and 0's in hex
def b2h(bstr):
        return ''.join( [ "%02X " % ord( x ) for x in bstr ] ).strip()

## Compress Callsign, Gridsquare and Powerlevel into ints

In [109]:
#call sign characters to ints: '0' to '9' = 0 to 9; 'A' to 'Z' = 10 to 35; space = 36
def cs_encode(c):
    if (c == ' '):
        return 36
    if (c<='9') and (c>='0'):
        return ord(c) - 48
    return ord(c) - ord('A') + 10

In [110]:
#a quick sanity check that things will go as expected
assert (cs_encode(' ') == 36)
assert (cs_encode('Z') == 35)
assert (cs_encode('9') == 9)

In [111]:
#compress call sign into an int
N = cs_ord(callsign[0])          #first character can take on any of the 37 values including [sp],
N = N * 36 + cs_encode(callsign[1]) #second character cannot then be a space so can have 36 values
N = N * 10 + cs_encode(callsign[2]) #third character must always be a number, so only 10 values are possible.
N = 27 * N + cs_encode(callsign[3]) - 10
N = 27 * N + cs_encode(callsign[4]) - 10 #Characters at the end cannot be numbers,
N = 27 * N + cs_encode(callsign[5]) - 10

In [112]:
print "N: " + str(N)

N: 145782806


In [113]:
#grid square *characters* to ints: 'A' to 'R' = 0 to 17
def gschar_encode(c):
    return ord(c) - 65

#grid square *ints* to ints: '0' to '9' = 0 to 9
def gsint_encode(c):
    return ord(c) - 48

In [114]:
#compress gridsquare and power level into int
M = (179 - 10 * gschar_encode(gridsquare[0]) - gsint_encode(gridsquare[2]) ) * 180 + 10 * gschar_encode(gridsquare[1]) + gsint_encode(gridsquare[3])
M = M * 128 + powerlevel + 64

In [115]:
print "M: " + str(M)

M: 3495380


## Create one message array for encoding

In [116]:
#concat the 28 bits in N and the 22 bits in M into 50 consecutive bits
#then add 31 empty bits that will be used (later) to flush the last msg bit through convolution shift register

msgN = bitarray("{:028b}".format(N)) #format N as 28 bits of binary
msgM = bitarray("{:022b}".format(M)) #format M as 22 bits of binary
flushbits = 31 * bitarray('0') #31 empty flush bits (used later as the 'tail' message during convolution encoding)

#concat the messages: N + M + flushbits
msg = bitarray() #empty bitarray
msg.extend(msgN)
msg.extend(msgM)
msg.extend(flushbits)

assert (msg.length() == 81) #msg must be 81 bits long

In [117]:
print "msg bytes: " + b2h(msg.tobytes())
print "bit lenght: " + str(msg.length())

msg bytes: 8B 07 81 6D 55 75 00 00 00 00 00
bit lenght: 81


## Convolution encoding!

In [118]:
#calc the partity bits for a shift register state
def paritybits(polylist, shiftreg):
    parity = []
    
    for poly in polylist:
        tmpreg = shiftreg & poly #for each tap, see if there is a bit set in the shift register
        p = int(tmpreg.count(1) % 2) #modulo2 addition of the tap values (number of bits set to "1" mod 2)
        parity.append(p)
        
    return parity #return as many parity bits as there are polynomials describing the shiftreg taps

In [119]:
#move the message one bit at a time into the shiftreg and calc parity for each shift
def encode(message, polylist):
    
    K = polylist[0].length() #constraint length (K) is the length of the polynomial(s) that describe the taps
    
    #create a shift register K in size, initialize to all zeros
    #str("{:0"+str(K)+"b}").format(0) means create a string, length K, in binary, representing zero 
    sr = bitarray(str("{:0"+str(K)+"b}").format(0))
    
    output = []
    
    #push each message bit into the shift register and generate parity bits
    for i in range(0, len(message)):
        
        #move the shift register one bit to the right
        for s in range(len(sr)-1, 0, -1):
            sr[s] = sr[s-1]
            
        #shift the next message bit into the shift register
        sr[0] = message[i]
        #print sr.to01() #debug the shift register as a string of 1's and 0's
        
        #generate the convolutional parity bits for this shift register state
        output.extend(paritybits(polylist, sr))

    return output

In [120]:
#encode our WSPR message!
#the "polynomials" represent the "taps" in the convolutional filter (specifies how parity bits are calculated)

poly0 = bitarray("{:032b}".format(0xf2d05351))
#WSPR moves shift register LSB to MSB first (shift left), the reverse of our approach, so flip the polys around
poly0.reverse() 

poly1 = bitarray("{:032b}".format(0xe4613c47))
#WSPR moves shift register LSB to MSB first (shift left), the reverse of our approach, so flip the polys around
poly1.reverse()

wspr_polynomials = [poly0, poly1]

print wspr_polynomials

[bitarray('10001010110010100000101101001111'), bitarray('11100010001111001000011000100111')]


In [121]:
convolvedmsg = encode(msg, wspr_polynomials)  #this is where all the work is done, using the two prev functions

print convolvedmsg
print
print "size: " + str(len(convolvedmsg))

[1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0]

size: 162


## Interleave Bits (make more robust to noise that isn't randomly distributed)

In [122]:
#permute the convolved message

#generate the permuted index ordering
permutedindex = []
for i in range(0, 256):
    #[::-1] is extended slice syntax to reverse string; int(str, 2) is binary string, base 2 to int
    newlocation = int("{:08b}".format(i)[::-1], 2)
    permutedindex.append(newlocation)

print permutedindex

[0, 128, 64, 192, 32, 160, 96, 224, 16, 144, 80, 208, 48, 176, 112, 240, 8, 136, 72, 200, 40, 168, 104, 232, 24, 152, 88, 216, 56, 184, 120, 248, 4, 132, 68, 196, 36, 164, 100, 228, 20, 148, 84, 212, 52, 180, 116, 244, 12, 140, 76, 204, 44, 172, 108, 236, 28, 156, 92, 220, 60, 188, 124, 252, 2, 130, 66, 194, 34, 162, 98, 226, 18, 146, 82, 210, 50, 178, 114, 242, 10, 138, 74, 202, 42, 170, 106, 234, 26, 154, 90, 218, 58, 186, 122, 250, 6, 134, 70, 198, 38, 166, 102, 230, 22, 150, 86, 214, 54, 182, 118, 246, 14, 142, 78, 206, 46, 174, 110, 238, 30, 158, 94, 222, 62, 190, 126, 254, 1, 129, 65, 193, 33, 161, 97, 225, 17, 145, 81, 209, 49, 177, 113, 241, 9, 137, 73, 201, 41, 169, 105, 233, 25, 153, 89, 217, 57, 185, 121, 249, 5, 133, 69, 197, 37, 165, 101, 229, 21, 149, 85, 213, 53, 181, 117, 245, 13, 141, 77, 205, 45, 173, 109, 237, 29, 157, 93, 221, 61, 189, 125, 253, 3, 131, 67, 195, 35, 163, 99, 227, 19, 147, 83, 211, 51, 179, 115, 243, 11, 139, 75, 203, 43, 171, 107, 235, 27, 155, 91, 

In [123]:
#use the permuted index to reorder the convolved message

interleavedmsg = [None] * 162  #initialize a list of 162 elements (like an array initialization)

nextconvolvedindex = 0
for i in range(0, len(permutedindex)):
    if permutedindex[i] < 162:
        
        interleavedmsg[permutedindex[i]] = convolvedmsg[nextconvolvedindex]
        nextconvolvedindex += 1
        
print interleavedmsg
print
print "size: " + str(len(interleavedmsg))

[1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1]

size: 162


## Add a sync bit to produce symbols for 4FSK

In [124]:
#combine each item in the syncvector with each item in the interleavedmsg

syncvector = [1,1,0,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,1,0,0,1,0,1,1,1,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,1,1,0,0,1,1,0,1,0,0,0,1,1,0,1,0,0,0,0,1,1,0,1,0,1,0,1,0,1,0,0,1,0,0,1,0,1,1,0,0,0,1,1,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,1,1,0,1,1,0,0,1,1,0,1,0,0,0,1,1,1,0,0,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,1,1,0,1,0,1,1,0,0,0,1,1,0,0,0]

txmsg = []
for i in range(0, len(syncvector)):
    txmsg.append(syncvector[i] + (interleavedmsg[i] * 2))

print "This is what we will transmit:"
print
print txmsg
print
print "size: " + str(len(txmsg))

This is what we will transmit:

[3, 1, 2, 2, 0, 2, 2, 2, 3, 0, 0, 0, 1, 3, 3, 0, 0, 0, 1, 2, 2, 3, 2, 3, 1, 3, 3, 0, 2, 0, 0, 2, 2, 2, 1, 2, 2, 1, 2, 3, 2, 2, 2, 2, 2, 2, 3, 2, 3, 3, 2, 2, 3, 3, 2, 1, 2, 0, 2, 3, 3, 0, 1, 0, 0, 0, 0, 3, 3, 2, 3, 0, 3, 0, 1, 2, 1, 2, 0, 3, 0, 2, 3, 0, 3, 3, 0, 0, 2, 3, 1, 2, 3, 0, 3, 0, 2, 0, 3, 0, 0, 2, 0, 2, 1, 0, 0, 1, 2, 0, 3, 3, 1, 0, 1, 3, 2, 0, 1, 3, 0, 1, 0, 0, 0, 1, 1, 1, 2, 2, 2, 0, 2, 1, 2, 3, 0, 0, 1, 3, 2, 2, 0, 0, 0, 2, 2, 3, 3, 2, 3, 0, 1, 3, 0, 2, 2, 3, 1, 0, 0, 2]

size: 162
